In [26]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os.path as op
import random
from data_prep import prepare_datasets, get_dfs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read Data

In [27]:
seed = 42
random.seed(seed)
np.random.seed(seed)

if op.exists(op.join("data", "train.csv")) and op.exists(op.join("data", "val.csv")) and op.exists(op.join("data", "test.csv")):
    train_df = pd.read_csv(op.join("data","train.csv"))
    val_df = pd.read_csv(op.join("data","val.csv"))
    test_df = pd.read_csv(op.join("data","test.csv"))
else:
    train_df, val_df, test_df = prepare_datasets(op.join("data","weather.csv"))
df_dct = {"train": train_df, "val": val_df, "test": test_df}
df_timed_dct = get_dfs(df_dct)


<ipython-input-27-edad8f1c45b8>:10: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  train_df, val_df, test_df = prepare_datasets(op.join("data","weather.csv"))


Cleaning Data


  0%|          | 145/83084 [00:00<00:57, 1444.36it/s]

Creating Dataframes


100%|██████████| 2170/2170 [00:01<00:00, 1335.25it/s]


Creating dataset for time_offset=5
Creating dataset for time_offset=10
Creating dataset for time_offset=15
Creating dataset for time_offset=30
Creating dataset for time_offset=60


In [80]:
df_dct["train"]

,M_SESSION_UID,M_WEATHER_FORECAST_SAMPLES_M_SESSION_TYPE,M_TRACK_ID,M_FORECAST_ACCURACY,M_WEATHER_FORECAST_SAMPLES_M_WEATHER_0,M_WEATHER_FORECAST_SAMPLES_M_TRACK_TEMPERATURE_0,M_WEATHER_FORECAST_SAMPLES_M_AIR_TEMPERATURE_0,M_TRACK_TEMPERATURE_CHANGE_0,M_AIR_TEMPERATURE_CHANGE_0,M_RAIN_PERCENTAGE_0,...,M_WEATHER_FORECAST_SAMPLES_M_AIR_TEMPERATURE_90,M_TRACK_TEMPERATURE_CHANGE_90,M_AIR_TEMPERATURE_CHANGE_90,M_RAIN_PERCENTAGE_90,M_WEATHER_FORECAST_SAMPLES_M_WEATHER_120,M_WEATHER_FORECAST_SAMPLES_M_TRACK_TEMPERATURE_120,M_WEATHER_FORECAST_SAMPLES_M_AIR_TEMPERATURE_120,M_TRACK_TEMPERATURE_CHANGE_120,M_AIR_TEMPERATURE_CHANGE_120,M_RAIN_PERCENTAGE_120
0,2.106082e+16,8.0,28.0,0.0,0.0,33.0,25.0,2.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.106082e+16,10.0,28.0,0.0,0.0,33.0,25.0,2.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.106082e+16,8.0,28.0,0.0,0.0,33.0,25.0,2.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.106082e+16,10.0,28.0,0.0,0.0,33.0,25.0,2.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.106082e+16,8.0,28.0,0.0,0.0,33.0,25.0,2.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83079,1.770268e+19,10.0,28.0,0.0,0.0,33.0,25.0,2.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83080,1.770268e+19,8.0,28.0,0.0,0.0,33.0,25.0,2.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83081,1.770268e+19,10.0,28.0,0.0,0.0,33.0,25.0,2.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83082,1.770268e+19,8.0,28.0,0.0,0.0,33.0,25.0,2.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
for time_offset in df_timed_dct:
    print(time_offset)
    print("train", df_timed_dct[time_offset]["train"].shape)
    print("val", df_timed_dct[time_offset]["val"].shape)
    print("test", df_timed_dct[time_offset]["test"].shape)
    print()

5
train (235874, 11)
val (287154, 11)
test (292581, 11)

10
train (152790, 11)
val (184360, 11)
test (187617, 11)

15
train (139412, 11)
val (163132, 11)
test (165306, 11)

30
train (138943, 11)
val (162031, 11)
test (164205, 11)

60
train (57887, 11)
val (68369, 11)
test (69456, 11)



## Weather

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, mean_absolute_percentage_error


def get_predict(dfs, model, problem_name, target_col, drop_cols, metric_func):
    X_train = dfs["train"].drop(drop_cols, axis=1)
    y_train = dfs["train"][target_col]
    X_val = dfs["val"].drop(drop_cols, axis=1)
    y_val = dfs["val"][target_col]
    X_test = dfs["test"].drop(drop_cols, axis=1)
    y_test = dfs["test"][target_col]

    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)
    y_pred_test = model.predict(X_test)
    print(problem_name)
    metric_func(y_train, y_val, y_test, y_pred_train, y_pred_val, y_pred_test)

def get_classification_metrics(y_train, y_val, y_test, y_pred_train, y_pred_val, y_pred_test):
    print("Train Acc: {}".format(accuracy_score(y_train, y_pred_train)))
    print("Train F1: {}".format(f1_score(y_train, y_pred_train, average=None)))
    print("Val Acc: {}".format(accuracy_score(y_val, y_pred_val)))
    print("Val F1: {}".format(f1_score(y_val, y_pred_val, average=None)))
    print("Test Acc: {}".format(accuracy_score(y_test, y_pred_test)))
    print("Test F1: {}".format(f1_score(y_test, y_pred_test, average=None)))
    print()

def get_regression_metrics(y_train, y_val, y_test, y_pred_train, y_pred_val, y_pred_test):
    print("Train MAE: {}".format(mean_absolute_error(y_train, y_pred_train)))
    print("Train MAPE: {}%".format(mean_absolute_percentage_error(y_train, y_pred_train) * 100))
    print("Val MAE: {}".format(mean_absolute_error(y_val, y_pred_val)))
    print("Val MAPE: {}%".format(mean_absolute_percentage_error(y_val, y_pred_val) * 100))
    print("Test MAE: {}".format(mean_absolute_error(y_test, y_pred_test)))
    print("Test MAPE Percantage: {}%".format(mean_absolute_percentage_error(y_test, y_pred_test) * 100))
    print()



In [ ]:
model = RandomForestClassifier(max_depth=10, random_state=0)
for time_offset in df_timed_dct:
    get_predict(df_timed_dct[time_offset], model, f"Weather {time_offset} Min", ["TARGET_WEATHER"], 
                ["TARGET_WEATHER","TARGET_RAIN_PERCENTAGE"], get_classification_metrics)

## RAIN_PERCENTAGE

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(max_depth=10, random_state=0)
for time_offset in df_timed_dct:
    get_predict(df_timed_dct[time_offset], model, f"Weather {time_offset} Min", ["TARGET_RAIN_PERCENTAGE"], 
                ["TARGET_WEATHER","TARGET_RAIN_PERCENTAGE"], get_regression_metrics)

## Class Distribution

In [ ]:
import matplotlib.pyplot as plt
plt.hist(y_train)
plt.show()

In [ ]:
plt.hist(y_test)
plt.show()

In [ ]:
set(y_test)